In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
OUTPUT_FOLDER = "/content/gdrive/MyDrive/Arecanut_Detection_with_ML/Classification/Trial_Classification/grayscale_feature/labelled_images"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

In [ ]:
# Loading stored reference features from CSV
def load_reference_features(csv_file):
    return pd.read_csv(csv_file, header=None).values.flatten()

In [ ]:
# Extracting features from an image
def extract_features(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not load image {image_path}")
        return None

    img = cv2.resize(img, (128, 128))  # Resize for consistency
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # GLCM Texture Features
    from skimage.feature import graycomatrix, graycoprops # Import graycomatrix and graycoprops
    glcm = graycomatrix(gray, distances=[5], angles=[0], symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]

    # Compute histogram (color features)
    hist = cv2.calcHist([img], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256]).flatten()

    # Edge detection (Canny)
    edges = cv2.Canny(gray, 100, 200)
    edge_density = np.sum(edges) / (128 * 128)

    # Combine features (texture + color + edge) to match the original feature extraction
    return np.hstack([contrast, correlation, energy, homogeneity, hist, edge_density])

In [ ]:
# Classifing and rename images
def classify_and_rename_images(image_folder, feature_csv, output_folder, threshold=0.8):
    # Loading stored features from CSV
    reference_features = load_reference_features(feature_csv)

    # Checking output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterating over all images in the folder
    for filename in os.listdir(image_folder):
        image_path = os.path.join(image_folder, filename)

        # Extracting features for the current image
        features = extract_features(image_path)
        if features is None:
            continue  # Skiping if feature extraction failed

        # Computing similarity (Cosine Similarity)
        similarity = np.dot(reference_features, features) / (np.linalg.norm(reference_features) * np.linalg.norm(features))

        # Assigning labels
        label = "1" if similarity >= threshold else "0"

        # Renaming and saving the image
        new_filename = f"{label}_{filename}"
        new_path = os.path.join(output_folder, new_filename)
        cv2.imwrite(new_path, cv2.imread(image_path))

        print(f"Processed: {filename} → {new_filename} (Similarity: {similarity:.2f})")

In [ ]:
image_folder = "/content/gdrive/MyDrive/Arecanut_Detection_with_ML/Classification/Trial_Classification/splitted_images"
feature_file = "/content/gdrive/MyDrive/Arecanut_Detection_with_ML/Classification/Trial_Classification/grayscale_feature/extracted_grayscale_feature.csv"
output_folder = "/content/gdrive/MyDrive/Arecanut_Detection_with_ML/Classification/Trial_Classification/grayscale_feature/labelled_images"

# Classify images
classify_and_rename_images(image_folder, feature_file, output_folder)

Processed: image_1_tile_0.png → 1_image_1_tile_0.png (Similarity: 0.95)
Processed: image_1_tile_1.png → 1_image_1_tile_1.png (Similarity: 0.95)
Processed: image_1_tile_2.png → 1_image_1_tile_2.png (Similarity: 0.87)
Processed: image_1_tile_3.png → 1_image_1_tile_3.png (Similarity: 0.82)
Processed: image_1_tile_4.png → 1_image_1_tile_4.png (Similarity: 0.91)
Processed: image_1_tile_5.png → 1_image_1_tile_5.png (Similarity: 0.88)
Processed: image_1_tile_6.png → 0_image_1_tile_6.png (Similarity: 0.70)
Processed: image_1_tile_7.png → 0_image_1_tile_7.png (Similarity: 0.80)
Processed: image_1_tile_8.png → 1_image_1_tile_8.png (Similarity: 0.93)
Processed: image_1_tile_9.png → 1_image_1_tile_9.png (Similarity: 0.96)
Processed: image_1_tile_10.png → 1_image_1_tile_10.png (Similarity: 0.96)
Processed: image_1_tile_11.png → 1_image_1_tile_11.png (Similarity: 0.86)
Processed: image_1_tile_12.png → 1_image_1_tile_12.png (Similarity: 0.84)
Processed: image_1_tile_13.png → 0_image_1_tile_13.png (Si

In [ ]:
!pip install scikit-image opencv-python pandas
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import graycomatrix, graycoprops
from sklearn.metrics import accuracy_score, classification_report
from google.colab import drive # Make sure this is imported

In [ ]:
# Function to check accuracy against manually labeled images
def check_accuracy(manual_folder, predicted_folder):
    # Lists to store true and predicted labels
    y_true = []
    y_pred = []

    # Get manually labeled images
    manual_files = {f for f in os.listdir(manual_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))}

    # Get predicted labeled images
    predicted_files = {f for f in os.listdir(predicted_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))}

    # Compare labels
    for manual_file in manual_files:
        # Extract true label from filename (assuming format: "1_image.png" or "0_image.png")
        true_label = manual_file.split('_')[0]
        original_name = '_'.join(manual_file.split('_')[1:])  # Get original filename without label

        # Find corresponding predicted file
        predicted_name = f"1_{original_name}" if f"1_{original_name}" in predicted_files else f"0_{original_name}"
        if predicted_name in predicted_files:
            pred_label = predicted_name.split('_')[0]
            y_true.append(int(true_label))
            y_pred.append(int(pred_label))
        else:
            print(f"Warning: No prediction found for {manual_file}")

    # Compute accuracy
    if y_true and y_pred:
        accuracy = accuracy_score(y_true, y_pred)
        report = classification_report(y_true, y_pred, target_names=['Negative (0)', 'Positive (1)'])
        print(f"Accuracy: {accuracy:.4f}")
        print("Classification Report:\n", report)
    else:
        print("Error: No matching labels found for comparison!")

In [ ]:
output_folder = "/content/gdrive/MyDrive/Arecanut_Detection_with_ML/Classification/Trial_Classification/grayscale_feature/labelled_images"
manual_folder = "/content/gdrive/MyDrive/Arecanut_Detection_with_ML/Classification/Trial_Classification/manually_labelled_images"

print("Manual folder path:", manual_folder)

# Check if the directory exists
if not os.path.exists(manual_folder):
  print(f"Error: Manual folder not found: {manual_folder}")

#Check accuracy against manually labeled images
check_accuracy(manual_folder, output_folder)

Manual folder path: /content/gdrive/MyDrive/AI_WRM/Classification/Trial1_Classification/manually_labelled_images
Accuracy: 0.7768
Classification Report:
               precision    recall  f1-score   support

Negative (0)       0.60      0.95      0.74        37
Positive (1)       0.96      0.69      0.81        75

    accuracy                           0.78       112
   macro avg       0.78      0.82      0.77       112
weighted avg       0.84      0.78      0.78       112

